<table style="margin: 0; text-align: left; background-color: #050A0E; border: 1px solid lightblue;">
    <tr>
        <td style="direction: rtl; text-align: right; color: #FFF8E7; ">
            <h2 style="color:#FFF8E7;">JobTonic</h2>
            <span style="color:#FFF8E7; font-size: 12px">
                כי חיפוש עבודה ראוי להרגיש כמו התחלה חדשה, לא כמו מסע מייגע.
                במקום להיסחף בגלים של מידע, 
                תנו לנו להיות הרוח שמכוונת את המפרש.
                הכלי
                 מזקק עבורכם את מהות כל משרה — בבהירות, במהירות, ובדיוק שמחזיר לכם שליטה.
                זה לא רק לחפש עבודה. זה לדייק מטרה. זה להתקדם עם ראש צלול, לב פתוח, וביטחון אמיתי.
                בכל שלב בדרך – אנחנו האנרגיה שמזיזה אתכם קדימה.
            </span>
        </td>
<td style="width: 290px; height: 200px; vertical-align: middle;">
            <img src="JobTonic.png" style="width: 100%; height: 100%; display: block;" />
        </td>
    </tr>
</table>

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import anthropic
import google.generativeai
import ollama
import json

In [2]:
# Constants
MODEL = "llama3.2"

# Load environment variables
load_dotenv(override=True)

# Initialize Google and Anthropic clients
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL_Claude = "claude-3-7-sonnet-latest"
claude = anthropic.Anthropic()

google_api_key = os.getenv('GOOGLE_API_KEY')
MODEL_Google = "gemini-2.0-flash"
google.generativeai.configure()

api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
deployment = os.getenv('DEPLOYMENT_4_1nano')

client = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=api_key,
    api_version=version
)

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
import fitz  # PyMuPDF

def read_pdf_text(filepath):
    try:
        text = ""
        with fitz.open(filepath) as doc:
            for page in doc:
                text += page.get_text()
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"
    
def read_website(url):
    try:
        website = Website(url)
        return website.text
    except Exception as e:
        return f"Error reading website: {e}"
    
def read_text_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        return text
    except Exception as e:
        return f"Error reading text file: {e}"

In [5]:
read_website_function = {
    "name": "read_website",
    "description": "Get the content of the website by its link. Call this whenever you need to read link content, for example when a customer provides a link to a job posting.",
    "parameters": {
        "type": "object",
        "properties": {
            "link": {
                "type": "string",
                "description": "The link to the website you want to read. For example: https://www.example.com/job/software-engineer",
            },
        },
        "required": ["link"],
        "additionalProperties": False
    }
}

In [6]:
read_website_function_antrophic = [
    {
        "name": "read_website",
        "description": "Get the content of the website by its link. Call this whenever you need to read link content, for example when a customer provides a link to a job posting.",
        "input_schema": {
            "type": "object",
            "properties": {
                "link": {
                    "type": "string",
                    "description": "The link to the website you want to read. For example: https://www.example.com/job/software-engineer",
                },
            },
            "required": ["link"],
        },
    }
]

In [7]:
def call_tool(reply, messages): 
    tool_responses = []
    messages.append(reply)

    for tool_call in reply.tool_calls:
        if tool_call.function.name == "read_website":
            try:
                # print(tool_call)
                arguments = json.loads(tool_call.function.arguments)
                # print(arguments)
                link = arguments.get('link')
                print(link)
                # Validate the link
                if not link or not link.startswith("http"):
                    raise ValueError(f"Invalid link: {link}")
                
                website_text = read_website(link)
                tool_responses.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": website_text
                })
                # print(messages)
            except Exception as e:
                # Handle errors gracefully
                error_message = f"Error processing tool_call_id {tool_call.id}: {e}"
                tool_responses.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": error_message
                })

        else:
            error_message = f"Error processing tool_call_id {tool_call.id}: Unsupported function {tool_call.function.name}"
            tool_responses.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": error_message
            })

    messages.extend(tool_responses)
    followup = client.chat.completions.create(
        model=deployment,
        messages=messages
    )

    # print("🔁 Model's final response:")
    # print(followup.choices[0].message.content)
    return followup.choices[0].message.content

In [14]:
# generic API call to 4 models
def callModel(company, messages):
    print(f"Calling {company}")
    if company == "ollama":
        response = ollama.chat(
            model=MODEL,
            messages=messages,
        )
        return response['message']['content']
    elif company == "openai":
        messages = messages
        tools = [{"type": "function", "function": read_website_function}]
        response = client.chat.completions.create(
            model=deployment,
            messages=messages,
            tools=tools
        )
        reply = response.choices[0].message
        if response.choices[0].finish_reason=="tool_calls":
            final_response = call_tool(reply, messages)
            return final_response
        else:
            return reply.content
    elif company == "anthropic":
        response = claude.messages.create(
            model=MODEL_Claude,
            max_tokens=500,
            temperature=0.4,
            system="you are a helpful assistant",
            tools=read_website_function_antrophic,
            tool_choice={"type": "auto"},
            messages=messages
        )
        return response.content[0].text
    elif company == "google":
        gemini = google.generativeai.GenerativeModel(
            model_name=MODEL_Google,
            system_instruction="You are a helpful assistant that can read and summarize website content.",
            )
        response = gemini.generate_content(messages)
        return response.text


In [18]:
text = read_text_file("prompt.txt")
messages_list = [
        {"role": "user", "content": text}
    ]
response = callModel("anthropic", messages_list)
print(response)

Calling anthropic
I'll conduct comprehensive research on generating passive income from knowledge-based products. To provide you with the most up-to-date and reliable information, I'll need to access various sources online. Let me start by examining some key resources on this topic.


In [19]:
messages_list.append({"role": "assistant", "content": response})
user_response = """
please find the relevant information by returning the links to the websites that contain the information you need to answer the question.
than use the read_website function to read the content of the website and answer the question.
"""
messages_list.append({"role": "user", "content": user_response})
response = callModel("anthropic", messages_list)
print(response)

Calling anthropic
I'll help you research passive income from knowledge-based products by finding relevant sources and analyzing them. Let me start by identifying some key websites that would contain valuable information on this topic.

Some relevant websites to explore would include:

1. https://www.smartpassiveincome.com/passive-income-ideas/
2. https://www.entrepreneur.com/money-finance/27-passive-income-ideas-you-can-start-today/435782
3. https://www.ramseysolutions.com/budgeting/how-to-create-passive-income
4. https://www.forbes.com/advisor/investing/passive-income-ideas/
5. https://www.coursera.org/articles/passive-income
6. https://www.shopify.com/blog/passive-income

Let me start by examining these sources to gather comprehensive information. I'll use the read_website function to access their content.


In [22]:
print(json.dumps(messages_list, indent=4, ensure_ascii=False))


[
    {
        "role": "user",
        "content": "Please conduct comprehensive, up-to-date, and reliable source-based research on the leading, most effective, and common ways to generate consistent, stable, and passive income from a knowledge-based product (personal, professional, or emotional knowledge) – without the need for ongoing, continuous work from the creator and with the goal of not disrupting family life and leisure balance.\n\nDuring the research, you are required to:\n\nRely on trustworthy, data-backed, and up-to-date sources (up to 2025 if possible) – including professional articles, academic research, market data, reputable industry blogs, success stories, and expert insights.\n\nReview various passive income models for knowledge products, such as: digital courses, e-books, knowledge-based applications, content memberships, subscription systems, digital partnerships, and more – detailing the advantages, disadvantages, and requirements of each model.\n\nPresent practica

In [23]:
messages_list.append({"role": "assistant", "content": response})
user_response = """
use the read_website function to read the content of the website and answer the question.
"""
messages_list.append({"role": "user", "content": user_response})
response = callModel("anthropic", messages_list)
print(response)

Calling anthropic
I'll use the read_website function to access the content of these websites and provide you with comprehensive information about generating passive income from knowledge-based products. Let me start with the first source.


In [24]:
print(json.dumps(messages_list, indent=4, ensure_ascii=False))


[
    {
        "role": "user",
        "content": "Please conduct comprehensive, up-to-date, and reliable source-based research on the leading, most effective, and common ways to generate consistent, stable, and passive income from a knowledge-based product (personal, professional, or emotional knowledge) – without the need for ongoing, continuous work from the creator and with the goal of not disrupting family life and leisure balance.\n\nDuring the research, you are required to:\n\nRely on trustworthy, data-backed, and up-to-date sources (up to 2025 if possible) – including professional articles, academic research, market data, reputable industry blogs, success stories, and expert insights.\n\nReview various passive income models for knowledge products, such as: digital courses, e-books, knowledge-based applications, content memberships, subscription systems, digital partnerships, and more – detailing the advantages, disadvantages, and requirements of each model.\n\nPresent practica